In [1]:
%matplotlib inline
%load_ext tensorboard

In [2]:
import gymnasium as gym

from stable_baselines3 import SAC, TD3

from gymnasium.envs.registration import register, registry
from stable_baselines3.common.evaluation import evaluate_policy
import time

import matplotlib
import matplotlib.pyplot as plt

import torch

2025-01-31 16:02:11.155902: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-01-31 16:02:11.168834: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1738339331.185863    6631 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1738339331.191073    6631 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-01-31 16:02:11.208381: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

In [3]:
if 'MarineEnv-v0' not in registry:
    register(
        id='MarineEnv-v0',
        entry_point='environments:MarineEnv',  # String reference to the class
    )

In [4]:
env_kwargs = dict(
    render_mode=None,
    continuous=True,
    # max_episode_steps=1200,
    training_stage=2,
    timescale=1/6,
    training=True,
)

In [5]:
env = gym.make('MarineEnv-v0', **env_kwargs)

In [6]:
sac_kwargs = {
    "policy": "MlpPolicy",  # Using MLP since ASV states are structured data
    "learning_rate": 3e-4,  # Stable baseline learning rate for SAC
    "buffer_size": int(1e6),  # Large buffer for off-policy learning
    "learning_starts": 10000,  # Start training after collecting enough data
    "batch_size": 256,  # Larger batch size stabilizes training
    "tau": 0.005,  # Polyak averaging coefficient for soft target updates
    "gamma": 0.99,  # Discount factor for long-term planning
    "train_freq": (1, "step"),  # Update every step
    "gradient_steps": 1,  # Number of gradient steps per update
    "action_noise": None,  # No action noise (SAC handles exploration via entropy)
    "replay_buffer_class": None,  # Use default SAC replay buffer
    "optimize_memory_usage": False,  # Use default memory mode
    "ent_coef": "auto_0.1",  # Automatic entropy tuning with initial value 0.1
    "target_update_interval": 1,  # Update target network every step
    "target_entropy": "auto",  # Automatic entropy tuning
    "use_sde": True,  # Use State-Dependent Exploration (better for continuous control)
    "sde_sample_freq": 64,  # Sample a new noise matrix every 64 steps
    "use_sde_at_warmup": True,  # Use SDE from the start
    "tensorboard_log": "./logging/",  # Log for debugging
    "policy_kwargs": {
        "net_arch": [256, 256],  # Sufficient for complex ASV decisions
        "activation_fn": torch.nn.ReLU,  # ReLU activation for stability
        "log_std_init": -2,  # Reduce initial log standard deviation
    },
    "verbose": 1,  # Show training progress
    "device": "auto",  # Use GPU if available
}


In [7]:
agent = SAC(
    env=env,
    **sac_kwargs,
)

Using cuda device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


In [13]:
agent.learn(total_timesteps=1e6, reset_num_timesteps=False, progress_bar=True, tb_log_name='sac')

Logging to ./logging/sac_0


Output()

---------------------------------
| rollout/           |          |
|    ep_len_mean     | 371      |
|    ep_rew_mean     | -447     |
| time/              |          |
|    episodes        | 2868     |
|    fps             | 38       |
|    time_elapsed    | 33       |
|    total_timesteps | 1001294  |
| train/             |          |
|    actor_loss      | 1.94e+04 |
|    critic_loss     | 1.04e+07 |
|    ent_coef        | 0        |
|    ent_coef_loss   | -6.3e+03 |
|    learning_rate   | 0.0003   |
|    n_updates       | 991293   |
|    std             | 1.89e+05 |
---------------------------------
----------------------------------
| rollout/           |           |
|    ep_len_mean     | 365       |
|    ep_rew_mean     | -434      |
| time/              |           |
|    episodes        | 2872      |
|    fps             | 42        |
|    time_elapsed    | 65        |
|    total_timesteps | 1002820   |
| train/             |           |
|    actor_loss      | 1.76e+04  |
|  

KeyboardInterrupt: 

In [10]:
eval_env = gym.make('MarineEnv-v0', **env_kwargs)
mean, std = evaluate_policy(model=agent, env=eval_env, n_eval_episodes=10, deterministic=True)
print(f'Mean: {mean:.2f}, Std: {std:.2f}')

/usr/local/lib/python3.11/dist-packages/stable_baselines3/common/evaluation.py:67: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  warnings.warn(


Mean: -333.27, Std: 341.03


In [8]:
%tensorboard --logdir ./logging/ --host=0.0.0.0

In [ ]:
agent.save('sac_3')
# agent = agent.load('sac_stage_2', device='cuda')

In [12]:
timescale = 1 / 6
for _ in range(10):
    env = gym.make('MarineEnv-v0', render_mode='human', continuous=True, training_stage=2, timescale=timescale, training=False)
    state, _ = env.reset()
    print(state)
    episode_rewards = 0 
    # flatten_state = flatten(env.observation_space, state)
    # state = torch.tensor(flatten_state, dtype=torch.float32, device=device).unsqueeze(0)
    for _ in range(int(400 / timescale)):
        action = agent.predict(state, deterministic=True)
        # print(action)
        # observation, reward, terminated, truncated, info = env.step((0, 0))
        observation, reward, terminated, truncated, info = env.step(action[0])
        env.render()
        # time.sleep(0.001)
        episode_rewards += reward
        print('===========================')
        print(observation)
        print(reward)
        
        if terminated or truncated:
            print(episode_rewards)
            break
    
        state = observation
            
    print(episode_rewards)
    print(state)
    env.close()

[116.932915   17.106941   13.836124   48.52811     1.8068955  48.52811
   0.          0.          0.          0.          0.          0.
   0.          0.          0.          0.       ]
[120.14634    17.187277   13.788364   48.134552   -1.4113989  48.361443
   0.          0.          0.          0.          0.          0.
   0.          0.          0.          0.       ]
0.21326666937934025
[116.932915   17.267612   13.74039    47.743916    1.8083245  48.194775
   0.          0.          0.          0.          0.          0.
   0.          0.          0.          0.       ]
0.21332627402411564
[117.083565   17.347948   13.692189   47.35611     1.6635118  48.028107
   0.          0.          0.          0.          0.          0.
   0.          0.          0.          0.       ]
0.513389057583279
[113.87014    17.428284   13.643921   46.97165     4.8942385  47.86144
   0.          0.          0.          0.          0.          0.
   0.          0.          0.          0.       ]
0.21

AttributeError: 'NoneType' object has no attribute 'fill'